# SIR Model for Viral Marketing

In [1]:
from decimal import Decimal
from datetime import timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
import pandas as pd
from tabulate import tabulate

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import clear_output

from SimCAD.configuration import Configuration
from SimCAD.configuration.utils import exo_update_per_ts, proc_trigger, bound_norm_random, \
    ep_time_step
from SimCAD.engine import ExecutionMode, ExecutionContext, Executor

In [2]:
sim_config = {
    'N': 1,
    'T': range(50)
}

seed = {}
env_processes = {}
initial_condition = {
    'Budget': float(50),
    'Ps': float(1000),
    'Pi': float(10),
    'Pr': float(0),
    'beta': float(0.05), # contact rate between S and I
    'gamma': float(0.20), # recover rate from I to R
    'timestamp': '2019-01-01 00:00:00'
}

# Parameters
epsilon = 0.03
subscription_fee = 1.0
incentive_cost = 10.0
stickiness_cost = 5.0
delta_beta = 0.1
delta_gamma = 0.1

ts_format = '%Y-%m-%d %H:%M:%S'
t_delta = timedelta(days=0, minutes=0, seconds=1)
def time_model(step, sL, s, _input):
    y = 'timestamp'
    x = ep_time_step(s, dt_str=s['timestamp'], fromat_str=ts_format, _timedelta=t_delta)
    return (y, x)

exogenous_states = exo_update_per_ts(
    {
    'timestamp': time_model
    }
)

In [3]:
# Behaviors choose actions depending on states
# 1) increase contact rate, beta (create incentive to spread)
# 2) decrease recover rate, gamma (improve stickiness)

def add_incentive(step, sL, s):
    delta = 0.0
    potential_delta = s['beta'] * delta_beta
    if (s['Ps'] > 3 * s['Pi']) and s['Budget'] > \
        abs(potential_delta * incentive_cost * s['Ps']):
        delta = potential_delta
    return {'delta': delta}

def add_stickiness(step, sL, s):
    delta = 0.0
    potential_delta = s['gamma'] * delta_gamma
    if (s['Pr'] > 2 * s['Pi']) and s['Budget'] > \
        abs(potential_delta * stickiness_cost * s['Pi']):
        delta = potential_delta
    return {'delta': delta}

def dummy_behavior(step, sL, s):
    return {'delta': 0.0}

# Mechanisms incur cost to modify beta or gamma
# 1) incur cost to create incentive to spread
# 2) incur cost to improve stickiness

def incur_incentive_cost(step, sL, s, _input):
    y = 'Budget'
    x = s['Budget'] - abs(_input['delta'] * s['Ps'] * incentive_cost)
    return (y, x)

def incur_stickiness_cost(step, sL, s, _input):
    y = 'Budget'
    x = s['Budget'] - abs(_input['delta'] * s['Pi'] * stickiness_cost)
    return (y, x)

def update_beta(step, sL, s, _input):
    y = 'beta'
    x = s['beta'] + _input['delta']
    return (y, x)

def update_gamma(step, sL, s, _input):
    y = 'gamma'
    x = s['gamma'] - _input['delta']
    return (y, x)

def S_model(step, sL, s, _input):
    y = 'Ps'
    x = s['Ps'] - s['beta'] * s['Ps']
    return (y, x)

def I_model(step, sL, s, _input):
    y = 'Pi'
    x = s['Pi'] + s['beta'] * s['Ps'] - s['gamma'] * s['Pi']
    return (y, x)
    
def R_model(step, sL, s, _input):
    y = 'Pr'
    x = s['Pr'] + s['gamma'] * s['Pi']
    return (y, x)

def collect_subscription(step, sL, s, _input):
    y = 'Budget'
    x = s['Budget'] + s['Pi'] * epsilon * subscription_fee
    return (y, x)

mechanisms = {
    'spread': {
        'behaviors': {
            'dummy': dummy_behavior
        },
        'states': {
            'Ps': S_model,
            'Pi': I_model,
            'Pr': R_model,
            'Budget': collect_subscription
        }      
    },
    'create_incentive': {
        'behaviors': {
            'action': add_incentive,
        },
        'states': {
            'beta': update_beta,
            'Budget': incur_incentive_cost,
        }
    },
    'improve_stickiness': {
        'behaviors': {
            'action': add_stickiness
        },
        'states': {
            'gamma': update_gamma,
            'Budget': incur_stickiness_cost,
        }
    }
}

In [4]:
def widget_handler(beta=float(0.05), gamma=float(0.20),
                   subscription_fee=float(1.0), 
                   incentive_cost=float(10.0), 
                   stickiness_cost=float(5.0)):
    initial_condition['beta'] = beta
    initial_condition['gamma'] = gamma
    subscription_fee = subscription_fee
    incentive_cost = incentive_cost
    stickiness_cost = stickiness_cost
    
    config = Configuration(
            sim_config=sim_config,
            state_dict=initial_condition,
            seed=seed,
            exogenous_states=exogenous_states,
            env_processes=env_processes,
            mechanisms=mechanisms)

    exec_mode = ExecutionMode()
    exec_context = ExecutionContext(exec_mode.single_proc)
    executor = Executor(exec_context, [config]) # Pass the configuration object inside an array
    raw_result, tensor = executor.main()
    df = pd.DataFrame(raw_result)
    df['timestamp'] = pd.to_datetime(df['timestamp'], format=ts_format)
    
    fig = plt.figure(figsize=(7, 14))
    
    sir = fig.add_subplot(3, 1, 1)
    sir.plot('timestamp', 'Ps', data=df, marker='', color='skyblue', linewidth=2)
    sir.plot('timestamp', 'Pi', data=df, marker='', color='orange', linewidth=2)
    sir.plot('timestamp', 'Pr', data=df, marker='', color='green', linewidth=2)
    sir.legend()
    
    beta_gamma = fig.add_subplot(3, 1, 2)
    beta_gamma.plot('timestamp', 'beta', data=df, marker='', color='skyblue', linewidth=2)
    beta_gamma.plot('timestamp', 'gamma', data=df, marker='', color='orange', linewidth=2)
    beta_gamma.legend()
    
    budget_pi = fig.add_subplot(3, 1, 3)
    budget_pi.plot('timestamp', 'Budget', data=df, marker='', color='skyblue', linewidth=2)
    budget_pi.plot('timestamp', 'Pi', data=df, marker='', color='orange', linewidth=2)
    budget_pi.legend()
    
    plt.show()
    
sliders = interact_manual(widget_handler, 
                beta=(0, 1, 0.01),
                gamma=(0, 1, 0.01),
                subscription_fee=(0, 10, 0.1),
                incentive_cost=(0, 20, 0.5),
                stickiness_cost=(0, 20, 0.5)
               )
sliders

<function __main__.widget_handler>

In [5]:
# config = Configuration(
#             sim_config=sim_config,
#             state_dict=initial_condition,
#             seed=seed,
#             exogenous_states=exogenous_states,
#             env_processes=env_processes,
#             mechanisms=mechanisms)

# from SimCAD.engine import ExecutionMode, ExecutionContext, Executor
# exec_mode = ExecutionMode()
# exec_context = ExecutionContext(exec_mode.single_proc)
# executor = Executor(exec_context, [config]) # Pass the configuration object inside an array
# raw_result, tensor = executor.main()
# df = pd.DataFrame(raw_result)

In [6]:
# df.plot('timestamp', ['Ps','Pi', 'Pr'])
# df.plot('timestamp', ['beta', 'gamma'])
# df.plot('timestamp', ['Budget', 'Pi'])
# df[['Ps','Pi', 'Pr']].describe()